In [1]:
from gensim.models import fasttext as ft
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import operator
import re

In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://fasttext.cc/docs/en/crawl-vectors.html
model=ft.load_facebook_model(os.path.join(os.getcwd(),"binaries/data/cc.en.300.bin"))

C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [4]:
window_size=3
stemmer = nltk.wordnet.WordNetLemmatizer()

In [32]:
def rank(text):
    text=text.lower()
    stops = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    stemmed_words = [stemmer.lemmatize(word) for word in tokens if word not in stops]
    filtered_words=[]
    for word in stemmed_words:
        filtered=re.sub('[^a-zA-Z]', '', word)
        if(len(filtered)>2):
            filtered_words.append(word)
    scores=dict()
    counts=dict()
    contextSizes=dict()
    for i in range(len(filtered_words)):
        counts.setdefault(filtered_words[i],0)
        counts[filtered_words[i]]+=1
        leftBound=max(0,i-window_size)
        rightBound=min(len(filtered_words)-1,i+window_size)
        contextSize=rightBound-leftBound+1
        if(filtered_words[i] in contextSizes or contextSize!=2*window_size+1):
            contextSizes.setdefault(filtered_words[i],contextSize)
            contextSizes[filtered_words[i]]=max(contextSizes[filtered_words[i]],contextSize)
        for j in range(i+1,rightBound+1):
            similarity_score=model.wv.similarity(filtered_words[i],filtered_words[j])
            similarity_score=(similarity_score+1)/2.0
            scores.setdefault(filtered_words[i],similarity_score)
            scores.setdefault(filtered_words[j],similarity_score)
            scores[filtered_words[i]]+=similarity_score#min(scores[filtered_words[i]],similarity_score)
            scores[filtered_words[j]]+=similarity_score#min(scores[filtered_words[j]],similarity_score)
    wordScores=list(map(list, scores.items()))
    for i in range(len(wordScores)):
        contextSize=contextSizes[wordScores[i][0]] if wordScores[i][0] in contextSizes else 2*window_size-1 
        wordScores[i][1]=wordScores[i][1]/(counts[wordScores[i][0]]*contextSize)
    wordScores=sorted(wordScores, key = lambda x: x[1])#,reverse=True)
    return wordScores